# Fine Tuning the COVID19 SIRD Model

Previously, a SIRD model was fit over current COVID19 Data. However, this fit displayed erratic behavior. We have several ways to address this.

* Using Interpolation rather than a high degree polynomial fit
* Changing the model to something that fits the attributes of COVID19:
    * The Exposed (Incubation Period)
    * Carrier classes
    * Seasonal virus?
    
Once this is fixed, the intent is to allow users to modify transitional functions to answer "what ifs".
* What if lockdown ends at this specific time?
* What if the travel ban gets lifted at this point?

## Interpolation of Transitional Functions
This includes $\beta(t)$, $\gamma(t)$, and $\delta(t)$. The issue with using a polynomial is that it must be true that 
$$\forall t\geq0, \beta(t),\gamma(t),\delta(t)\in[0,1].$$
The issue is that for any polynomial $p(t)$,
$$\lim_{t\to\infty}p(t)=\pm\infty$$
This causes very erratic behavior in out model. Instead we'll set interpolants to keep bounds. For the following, Hermite Cubic Interpolation with Chebyshev spacing will be used.

In [ ]:
"""
@author Vikram Ramavarapu
"""

import requests
import matplotlib.pyplot as plt
import matplotlib.axes as axis
import numpy as np
from scipy.integrate import odeint

#grab data from
"""
https://github.com/CSSEGISandData/COVID-19
https://github.com/pomber/covid19
"""
r = requests.get('https://pomber.github.io/covid19/timeseries.json')
json_data = r.json()

#number of days in record
days_ = len(json_data['China'])

"""
@param date_index the given number of days past Jan 22, 2020
@return number of infections on that given day
"""
def worldwide_i(date_index, country):
    if (country == 'worldwide'):
        infected = 0

        #loop through every country that isnt china and get (confirmed cases - (deaths+recoveries))
        for key in json_data:
            if (key != 'China'):
                infected = infected + (json_data[key][date_index]['confirmed'] - (json_data[key][date_index]['deaths'] + json_data[key][date_index]['recovered']))
        return infected
    else:
        return json_data[country][date_index]['confirmed'] - (json_data[country][date_index]['deaths'] + json_data[country][date_index]['recovered'])

"""
@param date_index the given number of days past Jan 22, 2020
@return number of recovered on that given day
"""
def worldwide_r(date_index, country):
    if (country == 'worldwide'):
        removed = 0
        for key in json_data:
            if (key != 'China'):
                removed = removed + json_data[key][date_index]['recovered']
        return removed
    else:
        return json_data[country][date_index]['recovered']

"""
@param date_index the given number of days past Jan 22, 2020
@return number of deaths on that given day
"""
def worldwide_d(date_index, country):
    if (country == 'worldwide'):
        removed = 0
        for key in json_data:
            if (key != 'China'):
                removed = removed + json_data[key][date_index]['deaths']
        return removed
    else:
        return json_data[country][date_index]['deaths']

def data(country):
    Is = []
    Rs = []
    Ds = []
    for i in range(days_):
        ret.append